<a href="https://colab.research.google.com/github/christopherjneelankavil/gpt_v1/blob/main/gpt_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64
batch_size = 128
max_iters = 3000
learing_rate = 3e-4
eval_iters = 100
eval_interval = 500
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

In [2]:
chars = ""

with open('/content/wizard_of_oz.txt' ,'r', encoding = 'utf-8') as f:
  text = f.read()
  chars = sorted(set(text))
vocab_size = len(chars)

In [3]:
string_to_int = {ch : i for i,ch in enumerate(chars)}
int_to_string = {i : ch for i,ch in enumerate(chars)}

encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [4]:
# train and val split

n = int(0.8 * len(data))

train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i + block_size + 1] for i in ix])

    x, y = x.to(device), y.to(device)
    return x, y

In [5]:
# estimate loss

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v

        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(*[Block(n_embd, n_head = 4, block_size = block_size) for _ in range(4)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)

    def forward(self, index, targets = None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, self.lm_head.out_features)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)

        return index

model = GPTLanguageModel(vocab_size)

In [7]:
# optimizer

model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr = learing_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.314, val loss: 4.313
step: 100, train loss: 2.223, val loss: 2.245
step: 200, train loss: 1.792, val loss: 1.842
step: 300, train loss: 1.585, val loss: 1.662
step: 400, train loss: 1.469, val loss: 1.572
step: 500, train loss: 1.379, val loss: 1.495
step: 600, train loss: 1.310, val loss: 1.457
step: 700, train loss: 1.261, val loss: 1.406
step: 800, train loss: 1.212, val loss: 1.396
step: 900, train loss: 1.182, val loss: 1.385
step: 1000, train loss: 1.148, val loss: 1.360
step: 1100, train loss: 1.119, val loss: 1.358
step: 1200, train loss: 1.091, val loss: 1.347
step: 1300, train loss: 1.068, val loss: 1.346
step: 1400, train loss: 1.041, val loss: 1.345
step: 1500, train loss: 1.017, val loss: 1.347
step: 1600, train loss: 0.992, val loss: 1.344
step: 1700, train loss: 0.972, val loss: 1.339
step: 1800, train loss: 0.948, val loss: 1.344
step: 1900, train loss: 0.926, val loss: 1.344
step: 2000, train loss: 0.907, val loss: 1.369
step: 2100, train loss: 0